In [ ]:
"""
Compute the CHSH S-parameter for a Bell state under various noise models.

Exercises:
- Replace the analytic scaling with simulated Kraus noise channels.
- Try amplitude damping and compare.
"""

In [9]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as AerEstimator  
from qiskit.transpiler import generate_preset_pass_manager   

backend = AerSimulator(method="statevector")
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
est = AerEstimator.from_backend(backend) 


def E(theta_a, theta_b):
    #correlator <psi| ZZ |psi> (= p_00 + p_11 - p_10 - p_01) with rotations by where |psi> = U(theta_a) \otimes U(theta_b)|Φ+>, that would be "the device preparation state"
    qc = QuantumCircuit(2)
    qc.h(0); qc.cx(0,1) #Bell state
    qc.ry(theta_a,0); qc.ry(theta_b,1) # modelize crudely the internal state of the device
    isa_qc = pm.run(qc)
    ob = SparsePauliOp("ZZ")
    isa_ZZ = ob.apply_layout(isa_qc.layout)
    pub = [(isa_qc,isa_ZZ)]
    return AerEstimator().run(pub).result()[0].data.evs

a0, a1 = 0, np.pi/2
b0, b1 = np.pi/4, -np.pi/4
S = E(a0,b0)+E(a0,b1)+E(a1,b0)-E(a1,b1)
if S >= 2:
    if abs(S - 2*np.sqrt(2)) < 1e-3:
        print("S =", S, " (Tsirelson bound!)")
    elif S < 2*np.sqrt(2):
        print("S =", S, " (CHSH violation!)")
    else:
        print("S =", S, " (error!)")
elif S < 2:
    print("S =", S, " (no CHSH violation)")
else:
    print("S =", S, " (error!)")

S = 2.8284271247461903  (Tsirelson bound!)
